![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [68]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


cc_apps = pd.read_csv("cc_approvals.data", header=None) 
print(cc_apps.head(100))

#1) Cleaning data
cc_apps_corrected=cc_apps.replace("?",np.NAN)
#2) Filling missing values 
def fill_missing_values(df):
    for column in df.columns:
        if df[column].dtypes== 'object':
           df[column]= df[column].fillna(
             df[column].value_counts().index[0] 
           )
        else:
           df[column]= df[column].fillna(df[column].mean())
    return df
    
cc_apps_corrected=fill_missing_values(cc_apps_corrected)
#3) changing values into numbers
cc_apps_clean=pd.get_dummies(cc_apps_corrected,drop_first=True)

print(cc_apps_clean.head(10))
#4) Defining dependent and independent variables
X=cc_apps_clean.iloc[:, :-1].values #all rows and coumns except the last one
y=cc_apps_clean.iloc[:, [-1]].values #all rows only in the last column

#5) Dividing intro trainning and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#6) Standarizing numeric data
scaler=StandardScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

#7) Creating the model
logreg = LogisticRegression()

#8)Traning the model
logreg.fit(rescaledX_train, y_train)

#9) Make the predictions
y_train_pred = logreg.predict(rescaledX_train)

#10) Print the confusion matrix of the logreg model
print(confusion_matrix(y_train, y_train_pred))

#11)Establishing the grid values 
tol = [0.01, 0.001, 0.0001]
max_iter = [120, 170, 200]

#12)reating a dictionary with the results
model_dictionary={"tol":tol,"max_iter":max_iter}

#13) It sets up the grid search process
grid_model = GridSearchCV(estimator=logreg, param_grid=model_dictionary, cv=5)

#14) Starts the process of training with the different combinations
grid_model_result = grid_model.fit(rescaledX_train, y_train)

#15) Results, saving into two variable names
best_train_score, best_train_params = grid_model_result.best_score_, grid_model_result.best_params_

#16)Pringting the best results
print("Best: %f using %s" % (best_train_score, best_train_params))

#17) Extracting the best model
best_model = grid_model_result.best_estimator_
best_score =  best_model.score(rescaledX_test, y_test)

#18 Printing the best score
print("Accuracy of logistic regression classifier: ", best_score)

print("Your model expects:", X_train.shape[1], "values per row.")


   0      1       2  3  4   5   6      7  8  9   10 11   12 13
0   b  30.83   0.000  u  g   w   v  1.250  t  t   1  g    0  +
1   a  58.67   4.460  u  g   q   h  3.040  t  t   6  g  560  +
2   a  24.50   0.500  u  g   q   h  1.500  t  f   0  g  824  +
3   b  27.83   1.540  u  g   w   v  3.750  t  t   5  g    3  +
4   b  20.17   5.625  u  g   w   v  1.710  t  f   0  s    0  +
.. ..    ...     ... .. ..  ..  ..    ... .. ..  .. ..  ... ..
95  a  28.58   3.540  u  g   i  bb  0.500  t  f   0  g    0  -
96  b  23.00   0.625  y  p  aa   v  0.125  t  f   0  g    1  -
97  b      ?   0.500  u  g   c  bb  0.835  t  f   0  s    0  -
98  a  22.50  11.000  y  p   q   v  3.000  t  f   0  g    0  -
99  a  28.50   1.000  u  g   q   v  1.000  t  t   2  g  500  -

[100 rows x 14 columns]
        2      7  10     12  0_b  1_15.17  ...  6_z  8_t  9_t  11_p  11_s  13_-
0   0.000  1.250   1      0    1        0  ...    0    1    1     0     0     0
1   4.460  3.040   6    560    0        0  ...    0    1   

In [69]:
# 19) Prediction using original data

# Creating a new data frame with the new values
new_input_df = pd.DataFrame([[
    'a', 10.83, 45.000, 'u', 'g', 'w', 'v', 2.250, 't', 't', 3, 'g', 560
]])

# Repeting same steps
new_input_df.columns = cc_apps.columns[:-1]

# Cleaning data
new_input_filled = fill_missing_values(new_input_df)
new_input_encoded = pd.get_dummies(new_input_filled, drop_first=True)

# Aligning columns with the trainning
new_input_aligned = new_input_encoded.reindex(columns=cc_apps_clean.columns[:-1], fill_value=0)

# Matching with the column names
new_input_aligned.columns = new_input_aligned.columns.astype(str)

# Escaling
new_input_scaled = scaler.transform(new_input_aligned)

# Predicting data
prediction = best_model.predict(new_input_scaled)
print("Prediction:", "Not Approved" if prediction[0] == 1 else "Approved")


Prediction: Not Approved
